In [1]:
import pandas as pd
import numpy as np
import math
from jellyfish import jaro_winkler
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
from py_common_subseq import find_common_subsequences
import numbers
import time
from collections import Counter 
from fuzzywuzzy import fuzz
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import gc

In [8]:
import io
import shutil

with io.open('all_recipients.txt', encoding='utf-8', errors='ignore') as source:
    with io.open('all_recipients_utf.txt', mode='w', encoding='utf-8') as target:
        shutil.copyfileobj(source,target)

In [ ]:
overall_time = time.time()

#define column names
org_id = 'RecipientID'
org_name = 'RecipientName'
org_address1 = 'addressline1txt'
org_city = 'citynm'
org_state = 'stateabbreviationcd'
org_zip = 'zip'
#org_phone = 'phone'
org_web = 'websiteaddresstxt'
#org_acronym = 'acronym'
#org_alt_name = 'alt_name'

#set parameters
chunk_size = 5000
token_match_min = 3 # minimum number of matched tokens to be considered a match
token_limiter = .995 # percent of non-single tokens to tokenize, where rare tokens are at the bottom and common at the top
unique_token_freq_max = 5 #threshold <= to a token is considered "unique" and links to these tokens are counted double
name_weight = .75 #note that this is really .75 * 4 because there are 4 org name simularity metrics
name_uniqueness_weight = 1.5
state_weight = 1
zip_weight = 1
phone_weight = 2
domain_weight = 2
name_score_min = 3
composite_score_min = 4 #minimum composite match score to be considered a match

start_time = time.time()
print "LOADING INITIAL DATAFRAMES..."

df = pd.read_table('all_recipients_utf.txt',keep_default_na=True,error_bad_lines=False)
df.sort_values(['websiteaddresstxt','addressline1txt','citynm','stateabbreviationcd','zip'],inplace=True)
df.drop_duplicates(subset='RecipientID',inplace=True)

df.rename(columns={org_id:'id',org_name:'org_name',org_address1:'org_address1',org_city:'org_city',org_state:'org_state',org_zip:'org_zip',org_web:'org_web'
                   #,org_phone:'org_phone',org_acronym:'org_acronym',org_alt_name:'org_alt_name'
                  }, inplace=True)

df = df.replace(np.nan, '', regex=True)

df.org_name = df.org_name.astype('str')
df.org_address1 = df.org_address1.astype('str')
df.org_city = df.org_city.astype('str')
df.org_state = df.org_state.astype('str')
df.org_zip = df.org_zip.astype('str')
#right_df.r_phone = right_df.r_phone.astype('str')
df.org_web = df.org_web.astype('str')

print("Dataframes loaded --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "PRE-PROCESSING..."
#normalize state codes
state_lkup = pd.read_csv('state_lkup.csv',keep_default_na=False)

from collections import defaultdict
state_dict = defaultdict(list)
for state, acronym in zip(state_lkup.state.values,state_lkup.acronym.values):
    state_dict[state].append(acronym)

df.org_state = df.org_state.replace(np.nan, '', regex=True).str.lower()
df.org_state = df.org_state.replace(state_dict)

#clean up non numeric characters in phones
#clean_phones = []
#df.org_phone = df.org_phone.astype('str')
#df.org_phone = df.org_phone.replace(np.nan, '', regex=True)
#for phone in df.org_phone:
#    clean_phones.append(re.sub('[^0-9]','', phone))
    
#df['clean_phone'] = clean_phones

#isolate domains from web URLs
domains = []
df.org_web = df.org_web.replace(np.nan, '', regex=True).str.lower()
for web in df.org_web:
    domains.append(web.split('//')[-1].split('/')[0].strip('www.'))
df['domain'] = domains

print("states, phones, domains normalized --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "TOKENIZING, IDENTIFYING CANDIDATE MATCH PAIRS..."

unique_token_columns = [ #tokens here circumvent the token_limiter which is cutting off the most commonly occuring tokens
    'org_state', 
    'org_zip'#,
    #'clean_phone'

]

delta_token_columns = [
    #'org_acronym',
    'org_name',
    #'org_alt_name',
    'org_address1',
    'org_city',
    'domain'
]

# lowercase the name and split on spaces, remove non-alphanumeric chars
def tokenize_name(name):
    if isinstance(name, basestring) is True:
        clean_name = ''.join(c if c.isalnum() else ' ' for c in name)
        return clean_name.lower().split()
    else:
        return name
    
unique_tokens = []    
for col in unique_token_columns:
    for word in df[col]:
        if isinstance(word, float) is False:
            unique_tokens.append(tokenize_name(str(word)))
            
unique_flat_list = [item for sublist in unique_tokens for item in sublist]

#instantiate counter and use to count word frequencies in flat list
u_cnt = Counter()
for token in unique_flat_list:
    u_cnt[token] += 1

u_cnt_dict = dict(u_cnt) #convert to dictionary

unique_tokens_df = pd.DataFrame(u_cnt_dict.items(), columns=['token', 'count'])
unique_tokens_df = unique_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

#consider waiting to do the count flag thing later, instead use some type of "token type" code
unique_token_flag = []
for index, value in enumerate(unique_tokens_df['count']):
    if value == 1:
        unique_token_flag.append(0)  #for any tokens occuring only once, we exclude
    else:
        unique_token_flag.append(1)

unique_tokens_df['flag'] = unique_token_flag        

all_other_words = []
for col in delta_token_columns:
    for word in df[col]:
        if isinstance(word, float) is False:
            all_other_words.append(tokenize_name(str(word)))
            
flat_list = [item for sublist in all_other_words for item in sublist] #flatten list so it can be counted

#instantiate counter and use to count word frequencies in flat list
cnt = Counter()
for token in flat_list:
    cnt[token] += 1

cnt_dict = dict(cnt) #convert to dictionary

main_tokens_df = pd.DataFrame(cnt_dict.items(), columns=['token', 'count'])
main_tokens_df = main_tokens_df.sort_values(by='count')  #sorting by count so that we can take the first x% of tokens by rare frequency

#wait to do count until joined with unique tokens?
main_token_flag = []
for index, value in enumerate(main_tokens_df['count']):
    if value == 1:
        main_token_flag.append(0)  #for any tokens occuring only once, we exclude
    elif index < int(main_tokens_df.shape[0] * token_limiter): #important line, we are cutting the top x% of frequently occuring tokens
        main_token_flag.append(1)
    else:
        main_token_flag.append(0)  #for the most common tokens, we exclude

main_tokens_df['flag'] = main_token_flag

all_tokens = pd.concat([unique_tokens_df, main_tokens_df])

all_tokens.drop('count',axis=1,inplace=True)
all_tokens['flag'] = all_tokens.flag.astype(int) #converting flags to int
tokens_dct = all_tokens.to_dict('split') #converting tokens_df to dictionary
tokens_dct=dict(tokens_dct['data']) #honestly can't remember why this works, something to do with conversion to dictionary

#preparing token_ids which will be used for joining left and right dfs
all_tokens.sort_values(by='flag',ascending=False,inplace=True)
all_tokens.sort_values(by='token',inplace=True)
all_tokens.drop_duplicates(subset='token',keep='first',inplace=True)
token_ids = all_tokens.index.get_level_values(0)
all_tokens['token_id'] = token_ids

all_tokens.drop('flag',axis=1,inplace=True)
all_tokens['token_id'] = all_tokens.token_id.astype(int)
token_id_dct = all_tokens.to_dict('split')
tokens_id_dct=dict(token_id_dct['data'])

print("token dictionary created --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
vocabulary = np.array([w for w, c in tokens_dct.items() if c ==1]) #this works even without the ==1 and I don't know why
cv = CountVectorizer( vocabulary=vocabulary)

#now we are ready to tokenize left and right dataframes
all_cols = unique_token_columns + delta_token_columns

left_frame_list = []
for colname in all_cols:
    tokenmapping = cv.fit_transform(df[colname])
    df_row, token_id = tokenmapping.nonzero()

    left_frame_list.append(pd.DataFrame(np.vstack([vocabulary[token_id], df['id'].values[df_row]]).T, columns = ['token', 'id']))

left_keyed = pd.concat(left_frame_list)
left_keyed.drop_duplicates()#inplace=True)
#removing duplicates inplace was giving me a very strange issue where a small percentage of token_ids would be excluded from the left_keyed index

#append token_id to token as this will be more efficient to join with
left_token_ids = []
for token in left_keyed.token:
    left_token_ids.append(tokens_id_dct[token])

left_keyed['token_id'] = left_token_ids
left_keyed.sort_values(by='token_id',inplace=True)
left_keyed.set_index('token_id',inplace=True)
left_keyed.drop('token',axis=1,inplace=True)

left_keyed.sort_values(by='id',inplace=True)

#right is just a copy of left
right_keyed = left_keyed.copy()

aggregations = {
    'id_l': 'count'
}

left_keyed.to_csv('left_keyed.csv')

del left_keyed
gc.collect()
left_keyed = pd.DataFrame()

print("tokens keyed to org records, now beginning chunking process to join left/right dfs --- %s seconds ---" % (time.time() - start_time))
start_time = time.time()

bonus_point_tokens = []
for token in main_tokens_df[(main_tokens_df['count'] > 1) & (main_tokens_df['count']<=unique_token_freq_max)].token:
    bonus_point_tokens.append(tokens_id_dct[token])

n=0
match_dfs = []
for chunk in pd.read_csv('left_keyed.csv',keep_default_na=False,chunksize=chunk_size,index_col='token_id'):
    
    n=n+1
    print n
    joined = chunk.join(right_keyed, how='inner',lsuffix='_l',rsuffix='_r')
    joined['id_l']=joined.id_l.astype('str')
    #double-counting unique token matches

    intersection_bonus_tokens = set(bonus_point_tokens).intersection(set(list(joined.index)))    

    bonus_token_joins = []
    for token_id in intersection_bonus_tokens:
        bonus_token_joins.append(joined.loc[token_id])

    bonus_joins = pd.concat(bonus_token_joins)
    bonus_joins_cols = bonus_joins[['id_l','id_r']].copy()
    bonus_joins_cols.dropna(inplace=True)
    joined_bonus = pd.concat([joined,bonus_joins_cols])
    
    keys_grouped = joined_bonus.groupby(by=['id_l', 'id_r']).agg(aggregations)
    keys_grouped.rename(columns={'id_l':'id_l count'}, inplace=True)
    matched_records = keys_grouped[keys_grouped['id_l count'] >= token_match_min]
    matched_records.reset_index(inplace=True)
    duplicate_candidates = matched_records[matched_records['id_l'] <> matched_records['id_r']]
    match_dfs.append(duplicate_candidates)
    del [[joined,bonus_joins,joined_bonus,keys_grouped,matched_records,duplicate_candidates]]
    gc.collect()
    joined = pd.DataFrame() #trying to preserve memory by returning variables to 0 mem usage
    bonus_joins = pd.DataFrame()
    joined_bonus = pd.DataFrame()
    keys_grouped = pd.DataFrame()
    matched_records = pd.DataFrame()
    duplicate_candidates = pd.DataFrame()
    
all_match_candidates = pd.concat(match_dfs)
del match_dfs
gc.collect()
match_dfs = pd.DataFrame()

all_match_candidates.reset_index(inplace=True)
all_match_candidates.drop(labels='index',axis=1,inplace=True)
all_match_candidates.id_l.astype('str')

print("match candidates identified --- %s seconds ---" % (time.time() - start_time))

start_time = time.time()
print "REMOVING REDUNDANT DUPLICATE ID PAIRS..."

match_tuples = list(zip(all_match_candidates['id_l'], all_match_candidates['id_r']))

sorted_match_tuples = []
for tup in match_tuples:
    s = tuple(sorted(tup))
    sorted_match_tuples.append(s)

all_match_candidates.drop(['id_l','id_r'],axis=1,inplace=True)
all_match_candidates['id_tuples'] = sorted_match_tuples

new_col_list = ['id_l','id_r']
for n,col in enumerate(new_col_list):
    all_match_candidates[col] = all_match_candidates['id_tuples'].apply(lambda location: location[n])
    
all_match_candidates.drop('id_tuples',axis=1,inplace=True)
all_match_candidates.drop_duplicates(inplace=True)

print("redundant id pairs removed --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "MERGING MATCH CANDIDATES WITH ORIGINAL DATA..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

right_df = df.copy()
df.rename(columns={'id':'id_l','org_name':'l_org_name','org_address1':'l_address1','org_city':'l_city','org_state':'l_state','org_zip':'l_postal_code'
                   #,'clean_phone':'l_clean_phone'
                   ,'domain':'l_domain'}, inplace=True)
right_df.rename(columns={'id':'id_r','org_name':'r_org_name','org_address1':'r_address1','org_city':'r_city','org_state':'r_state','org_zip':'r_postal_code'
                         #,'clean_phone':'r_clean_phone'
                         ,'domain':'r_domain'}, inplace=True)

left_match_data = df[['id_l','l_org_name','l_city','l_state','l_postal_code','l_domain'
                      #,'l_clean_phone'
                     ]].copy()
right_match_data = right_df[['id_r','r_org_name','r_city','r_state','r_postal_code','r_domain'
                             #,'r_clean_phone'
                            ]].copy()

#making sure keys are str, results in blank df otherwise
left_match_data.id_l = left_match_data.id_l.astype('str')
right_match_data.id_r = right_match_data.id_r.astype('str')
all_match_candidates.id_l = all_match_candidates.id_l.astype('str')
all_match_candidates.id_r = all_match_candidates.id_r.astype('str')

#merging matched_records df with original record data for ease of review
l_conc = pd.merge(all_match_candidates, left_match_data, on='id_l')
full_conc = pd.merge(l_conc, right_match_data, on='id_r')

print("original data merged with matches --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "SCORING ORG NAME SIMULARITY..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on edit distance of org names
def jaro_simularity(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return jaro_winkler(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '')
def fuzz_partial(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.partial_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_sort(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_sort_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_set(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_set_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)

full_conc['l_org_name'] = full_conc['l_org_name'].replace('', 'none', regex=True).astype('str')
full_conc['r_org_name'] = full_conc['r_org_name'].replace('', 'none', regex=True).astype('str')

jaro_time = time.time()
full_conc['jaro_score'] = full_conc.apply(lambda x: jaro_simularity(x.l_org_name, x.r_org_name), axis=1)
print("jaro scores done --- %s seconds ---" % (time.time() - jaro_time))
partial_time = time.time()
full_conc['fuzz_partial_score'] = full_conc.apply(lambda x: fuzz_partial(x.l_org_name, x.r_org_name), axis=1)
print("fuzz partial scores done --- %s seconds ---" % (time.time() - partial_time))
sort_time = time.time()
full_conc['fuzz_sort_score'] = full_conc.apply(lambda x: fuzz_sort(x.l_org_name, x.r_org_name), axis=1)
print("fuzz sort scores done --- %s seconds ---" % (time.time() - sort_time))
set_time = time.time()
full_conc['fuzz_set_score'] = full_conc.apply(lambda x: fuzz_set(x.l_org_name, x.r_org_name), axis=1)
print("fuzz set scores done --- %s seconds ---" % (time.time() - set_time))
print ""

print("name simularity scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "SCORING NAME SEQUENCE UNIQUENESS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

org_tokens = []    
for word in df['l_org_name']:
    if isinstance(word, float) is False:
        org_tokens.append(tokenize_name(str(word)))
            
org_flat_list = [item for sublist in org_tokens for item in sublist]

#instantiate counter and use to count word frequencies in flat list
org_cnt = Counter()
for token in org_flat_list:
    org_cnt[token] += 1

org_cnt_dict = dict(org_cnt) #convert to dictionary

def sequence_uniqueness(seq):
    return sum(1/org_cnt_dict[str.lower(t)]**0.5 for t in seq)

def name_similarity(a, b):
    if isinstance(a,basestring) is True and isinstance(b,basestring) is True:
        if len(a) > 1 and len(b) > 1:
            a_tokens = set(tokenize_name(a))
            b_tokens = set(tokenize_name(b))
            a_uniq = sequence_uniqueness(a_tokens)
            b_uniq = sequence_uniqueness(b_tokens)

            return sequence_uniqueness(a_tokens.intersection(b_tokens))/(a_uniq * b_uniq) ** 0.5
        else:
            return 0
    else:
        return 0

full_conc['uniq'] = full_conc.apply(lambda x: name_similarity(x.l_org_name, x.r_org_name), axis=1)

print("name uniqueness scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR STATE CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def sanitize_state(state):
    if isinstance(state,basestring) is True:
        return ''.join(c for c in (state or '') if c in 'abcdefghijklmnopqrstuvwxyz')
    else:
        return ''
    
def state_match(state_a, state_b):
    sanitized_state_a = str(sanitize_state(state_a))
    sanitized_state_b = str(sanitize_state(state_b))

    # if the value is too short, means it's fubar
    if len(sanitized_state_a) < 2 or len(sanitized_state_b) < 2:
        return 0
    if state_a == state_b:
        return 1
    else:
        return 0    

full_conc['state_match'] = full_conc.apply(lambda x: state_match(x.l_state, x.r_state), axis=1)

print("state codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR POSTAL CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching postal code

def sanitize_postal(postal):
    if isinstance(postal, basestring) is True:
        return ''.join(c for c in (postal or '') if c in '1234567890')
    if isinstance(postal, float) is False:
        return postal

def postal_simularity(postal_a, postal_b):
    sanitized_postal_a = str(sanitize_postal(postal_a))
    sanitized_postal_b = str(sanitize_postal(postal_b))

    # if the number is too short, means it's fubar
    if len(sanitized_postal_a) < 5 or len(sanitized_postal_b) < 5:
        return 0
    if float(max(len(sub) for sub in find_common_subsequences(sanitized_postal_a, sanitized_postal_b))) / 5 >= 1:
        return 1
    else:
        return 0
    
full_conc['zip_match'] = full_conc.apply(lambda x: postal_simularity(x.l_postal_code, x.r_postal_code), axis=1)
    
print("postal codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR WEB DOMAIN MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def domain_match(domain_a, domain_b):
    if isinstance(domain_a, basestring) is True and isinstance(domain_b, basestring) is True:
        if len(domain_a) > 4 and len(domain_b) > 4:
            if domain_a == domain_b:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0


full_conc['domain_match'] = full_conc.apply(lambda x: domain_match(x.l_domain, x.r_domain), axis=1)

print("web domains checked --- %s seconds ---" % (time.time() - start_time))
print ""

#start_time = time.time()
#print "CHECKING FOR PHONE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching phone
#def phone_simularity(phone_a, phone_b):

#    if len(phone_a) < 10 or len(phone_b) < 10:
#        return 0
#    elif float(max(len(sub) for sub in find_common_subsequences(phone_a, phone_b))) / 10 >= 1:
#        return 1
#    else:
#        return 0
    
#full_conc['phone_match'] = full_conc.apply(lambda x: phone_simularity(x.clean_phone, x.r_clean_phone), axis=1)
    
#print("phones checked --- %s seconds ---" % (time.time() - start_time))
#print ""

start_time = time.time()
print "COMPOSITE SCORING, PREDICTING MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#full_conc['overall_name_score'] = full_conc.jaro_score * name_weight \
#+ full_conc.fuzz_partial_score * name_weight \
#+ full_conc.fuzz_sort_score * name_weight \
#+ full_conc.fuzz_set_score * name_weight \
#+ full_conc.uniq * name_uniqueness_weight

#calculate composite match score based on component scores and weights
#full_conc['composite_match_score'] = full_conc.overall_name_score \
#+ full_conc.zip_match * zip_weight \
#+ full_conc.state_match * state_weight \
#+ full_conc.domain_match * domain_weight \
#+ full_conc.phone_match * phone_weight

#use labeled matches to train logistic regression to predict matches
training_data = pd.read_table('training_data3.txt')

feature_cols = ['jaro_score',
                'fuzz_partial_score',
                'fuzz_sort_score',
                'fuzz_set_score',
                'uniq',
                'state_match',
                'zip_match',
                'domain_match']#,
                #'phone_match'
        

# define X and y
X = training_data[feature_cols]
y = training_data['is_match']

# logistic regression
log = LogisticRegression()
log.fit(X, y)
y_pred_class = log.predict(full_conc[feature_cols])
y_pred_proba = log.predict_proba(full_conc[feature_cols])[:,1]

full_conc['match_pred'] = y_pred_class
full_conc['pred_proba'] = y_pred_proba

#we take any matches meeting either name match threshold or composite match threshold as matches for review
#org_matches = full_conc[(full_conc.overall_name_score >= name_score_min) | (full_conc.composite_match_score >= composite_score_min)]
org_matches = full_conc[full_conc.match_pred == 1]
failed_matches_for_review = full_conc[(full_conc.match_pred == 0) & (full_conc.pred_proba > .4)]

print("final matches isolated --- %s seconds ---" % (time.time() - start_time))
print ""

print("TOTAL COMPUTE TIME --- %s seconds ---" % (time.time() - overall_time))

org_matches.sort_values(by='pred_proba', ascending=True)

LOADING INITIAL DATAFRAMES...


Skipping line 1329580: expected 7 fields, saw 12
Skipping line 1385100: expected 7 fields, saw 8

C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dataframes loaded --- 20.8509998322 seconds ---

PRE-PROCESSING...
states, phones, domains normalized --- 15.4089999199 seconds ---

TOKENIZING, IDENTIFYING CANDIDATE MATCH PAIRS...
token dictionary created --- 93.5170001984 seconds ---
tokens keyed to org records, now beginning chunking process to join left/right dfs --- 134.623999834 seconds ---
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
19

In [7]:
full_conc.to_csv('full_conc for grant recipients.csv')

In [2]:
full_conc = pd.read_csv('full_conc for grant recipients.csv')
full_conc.head()



C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,id_l count,id_l,id_r,l_org_name,l_city,l_state,l_postal_code,l_domain,r_org_name,r_city,r_state,r_postal_code,r_domain
0,0,3,10,100949,ONE HEART CHURCH,STOCKBRIDGE,ga,30281.0,NaN,TLC Pet Rescue,Stockbridge,ga,30281.0,NaN
1,1,3,10,106556,ONE HEART CHURCH,STOCKBRIDGE,ga,30281.0,NaN,AMBASSADOR LIFE CENTER INC,Stockbridge,ga,30281.0,NaN
2,2,3,100949,106556,TLC Pet Rescue,Stockbridge,ga,30281.0,NaN,AMBASSADOR LIFE CENTER INC,Stockbridge,ga,30281.0,NaN
3,3,3,10,121141,ONE HEART CHURCH,STOCKBRIDGE,ga,30281.0,NaN,HANDS OF HOPE CLINIC INC,STOCKBRIDGE,ga,30281.0,handsofhopeclinic.org
4,4,3,100949,121141,TLC Pet Rescue,Stockbridge,ga,30281.0,NaN,HANDS OF HOPE CLINIC INC,STOCKBRIDGE,ga,30281.0,handsofhopeclinic.org


In [3]:
#scoring match candidates based on edit distance of org names
def jaro_simularity(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return jaro_winkler(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '')
def fuzz_partial(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.partial_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_sort(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_sort_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)
def fuzz_set(left_record, right_record):
    if len(left_record) > 0 and len(right_record) > 0:
        if isinstance(left_record, numbers.Integral) is False and isinstance(right_record, numbers.Integral) is False:
            return fuzz.token_set_ratio(unicode(left_record, 'utf-8') or '', unicode(right_record, 'utf-8') or '') / float(100)

full_conc['l_org_name'] = full_conc['l_org_name'].replace('', 'none', regex=True).astype('str')
full_conc['r_org_name'] = full_conc['r_org_name'].replace('', 'none', regex=True).astype('str')

set_time = time.time()
full_conc['fuzz_set_score'] = full_conc.apply(lambda x: fuzz_set(x.l_org_name, x.r_org_name), axis=1)
full_conc.to_csv('full_conc all name scores done.csv')

In [2]:
full_conc = pd.read_csv('full_conc grant recipients up to fuzz_set_score.csv')

C:\Users\dbell\AppData\Local\Continuum\anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2718: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
df.head()

,RecipientID,RecipientName,addressline1txt,citynm,stateabbreviationcd,zip,websiteaddresstxt
0,1,ST JAMES EPISCOPAL CHURCH,1 NORTH CHARLES STREET,BALTIMORE,MD,21201.0,NaN
1,1,ST JAMES EPISCOPAL CHURCH,1001 DAVIS STREET,DUBUQUE,IA,52001.0,NaN
2,1,ST JAMES EPISCOPAL CHURCH,1019 S COLLEGE STREET,GEORGETOWN,TX,78626.0,NaN
3,1,ST JAMES EPISCOPAL CHURCH,10345 W Pico Boulevard,Los Angeles,CA,90064.0,NaN
4,1,ST JAMES EPISCOPAL CHURCH,1103 40TH STREET,ROCK ISLAND,IL,61201.0,NaN


In [5]:
full_conc = pd.read_csv('full_conc reduced to 1 percent proba after name scores.csv')
df = pd.read_table('all_recipients_utf.txt',keep_default_na=True,error_bad_lines=False)

def tokenize_name(name):
    if isinstance(name, basestring) is True:
        clean_name = ''.join(c if c.isalnum() else ' ' for c in name)
        return clean_name.lower().split()
    else:
        return name

start_time = time.time()
print "SCORING NAME SEQUENCE UNIQUENESS..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

org_tokens = []    
for word in df['RecipientName']:
    if isinstance(word, float) is False:
        org_tokens.append(tokenize_name(str(word)))
            
org_flat_list = [item for sublist in org_tokens for item in sublist]

#instantiate counter and use to count word frequencies in flat list
org_cnt = Counter()
for token in org_flat_list:
    org_cnt[token] += 1

org_cnt_dict = dict(org_cnt) #convert to dictionary

def sequence_uniqueness(seq):
    return sum(1/org_cnt_dict[str.lower(t)]**0.5 for t in seq)

def name_similarity(a, b):
    if isinstance(a,basestring) is True and isinstance(b,basestring) is True:
        if len(a) > 1 and len(b) > 1:
            a_tokens = set(tokenize_name(a))
            b_tokens = set(tokenize_name(b))
            a_uniq = sequence_uniqueness(a_tokens)
            b_uniq = sequence_uniqueness(b_tokens)

            return sequence_uniqueness(a_tokens.intersection(b_tokens))/(a_uniq * b_uniq) ** 0.5
        else:
            return 0
    else:
        return 0

full_conc['uniq'] = full_conc.apply(lambda x: name_similarity(x.l_org_name, x.r_org_name), axis=1)

print("name uniqueness scored --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR STATE CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def sanitize_state(state):
    if isinstance(state,basestring) is True:
        return ''.join(c for c in (state or '') if c in 'abcdefghijklmnopqrstuvwxyz')
    else:
        return ''
    
def state_match(state_a, state_b):
    sanitized_state_a = str(sanitize_state(state_a))
    sanitized_state_b = str(sanitize_state(state_b))

    # if the value is too short, means it's fubar
    if len(sanitized_state_a) < 2 or len(sanitized_state_b) < 2:
        return 0
    if state_a == state_b:
        return 1
    else:
        return 0    

full_conc['state_match'] = full_conc.apply(lambda x: state_match(x.l_state, x.r_state), axis=1)

print("state codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR POSTAL CODE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching postal code

def sanitize_postal(postal):
    if isinstance(postal, basestring) is True:
        return ''.join(c for c in (postal or '') if c in '1234567890')
    if isinstance(postal, float) is False:
        return postal

def postal_simularity(postal_a, postal_b):
    sanitized_postal_a = str(sanitize_postal(postal_a))
    sanitized_postal_b = str(sanitize_postal(postal_b))

    # if the number is too short, means it's fubar
    if len(sanitized_postal_a) < 5 or len(sanitized_postal_b) < 5:
        return 0
    if float(max(len(sub) for sub in find_common_subsequences(sanitized_postal_a, sanitized_postal_b))) / 5 >= 1:
        return 1
    else:
        return 0
    
full_conc['zip_match'] = full_conc.apply(lambda x: postal_simularity(x.l_postal_code, x.r_postal_code), axis=1)
    
print("postal codes checked --- %s seconds ---" % (time.time() - start_time))
print ""

start_time = time.time()
print "CHECKING FOR WEB DOMAIN MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

def domain_match(domain_a, domain_b):
    if isinstance(domain_a, basestring) is True and isinstance(domain_b, basestring) is True:
        if len(domain_a) > 4 and len(domain_b) > 4:
            if domain_a == domain_b:
                return 1
            else:
                return 0
        else:
            return 0
    else:
        return 0


full_conc['domain_match'] = full_conc.apply(lambda x: domain_match(x.l_domain, x.r_domain), axis=1)

print("web domains checked --- %s seconds ---" % (time.time() - start_time))
print ""

#start_time = time.time()
#print "CHECKING FOR PHONE MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#scoring match candidates based on matching phone
#def phone_simularity(phone_a, phone_b):

#    if len(phone_a) < 10 or len(phone_b) < 10:
#        return 0
#    elif float(max(len(sub) for sub in find_common_subsequences(phone_a, phone_b))) / 10 >= 1:
#        return 1
#    else:
#        return 0
    
#full_conc['phone_match'] = full_conc.apply(lambda x: phone_simularity(x.clean_phone, x.r_clean_phone), axis=1)
    
#print("phones checked --- %s seconds ---" % (time.time() - start_time))
#print ""

start_time = time.time()
print "COMPOSITE SCORING, PREDICTING MATCHES..." #<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#full_conc['overall_name_score'] = full_conc.jaro_score * name_weight \
#+ full_conc.fuzz_partial_score * name_weight \
#+ full_conc.fuzz_sort_score * name_weight \
#+ full_conc.fuzz_set_score * name_weight \
#+ full_conc.uniq * name_uniqueness_weight

#calculate composite match score based on component scores and weights
#full_conc['composite_match_score'] = full_conc.overall_name_score \
#+ full_conc.zip_match * zip_weight \
#+ full_conc.state_match * state_weight \
#+ full_conc.domain_match * domain_weight \
#+ full_conc.phone_match * phone_weight

#use labeled matches to train logistic regression to predict matches
training_data = pd.read_table('training_data3.txt')

feature_cols = ['id_l count',
                'jaro_score',
                'fuzz_partial_score',
                'fuzz_sort_score',
                'fuzz_set_score',
                'uniq',
                'state_match',
                'zip_match',
                'domain_match']#,
                #'phone_match'
        

# define X and y
X = training_data[feature_cols]
y = training_data['is_match']

# logistic regression
log = LogisticRegression()
log.fit(X, y)
y_pred_class = log.predict(full_conc[feature_cols])
y_pred_proba = log.predict_proba(full_conc[feature_cols])[:,1]

full_conc['match_pred'] = y_pred_class
full_conc['pred_proba'] = y_pred_proba

#we take any matches meeting either name match threshold or composite match threshold as matches for review
#org_matches = full_conc[(full_conc.overall_name_score >= name_score_min) | (full_conc.composite_match_score >= composite_score_min)]
org_matches = full_conc[full_conc.match_pred == 1]
failed_matches_for_review = full_conc[(full_conc.match_pred == 0) & (full_conc.pred_proba > .4)]

print("final matches isolated --- %s seconds ---" % (time.time() - start_time))
print ""

print("TOTAL COMPUTE TIME --- %s seconds ---" % (time.time() - overall_time))

org_matches.sort_values(by='pred_proba', ascending=True)

Skipping line 1329580: expected 7 fields, saw 12
Skipping line 1385100: expected 7 fields, saw 8



SCORING NAME SEQUENCE UNIQUENESS...
name uniqueness scored --- 1801.4230001 seconds ---

CHECKING FOR STATE CODE MATCHES...
state codes checked --- 1193.46700001 seconds ---

CHECKING FOR POSTAL CODE MATCHES...
postal codes checked --- 1091.80100012 seconds ---

CHECKING FOR WEB DOMAIN MATCHES...
web domains checked --- 1064.63000011 seconds ---

COMPOSITE SCORING, PREDICTING MATCHES...
final matches isolated --- 23.4989998341 seconds ---



NameError: name 'overall_time' is not defined

In [24]:
org_matches.shape

(104250, 27)

In [20]:
org_matches.shape

(54141, 27)

In [25]:
org_matches.to_csv('grant recipient dups4 excluding domain, including l_count.csv')

In [23]:
training_data = pd.read_table('training_data3.txt')

feature_cols = ['id_l count',
    'jaro_score',
                'fuzz_partial_score',
                'fuzz_sort_score',
                'fuzz_set_score',
                'uniq',
                'state_match',
                'zip_match']#,
                #'domain_match']#,
                #'phone_match'
        

# define X and y
X = training_data[feature_cols]
y = training_data['is_match']

# logistic regression
log = LogisticRegression()
log.fit(X, y)
y_pred_class = log.predict(full_conc[feature_cols])
y_pred_proba = log.predict_proba(full_conc[feature_cols])[:,1]

full_conc['match_pred'] = y_pred_class
full_conc['pred_proba'] = y_pred_proba

#we take any matches meeting either name match threshold or composite match threshold as matches for review
#org_matches = full_conc[(full_conc.overall_name_score >= name_score_min) | (full_conc.composite_match_score >= composite_score_min)]
org_matches = full_conc[full_conc.match_pred == 1]
failed_matches_for_review = full_conc[(full_conc.match_pred == 0) & (full_conc.pred_proba > .4)]

In [4]:
full_conc[full_conc.pred_proba < .01].shape

(35644204, 22)

In [22]:
from sklearn import cross_validation

feature_cols = ['jaro_score',
                'fuzz_partial_score',
                'fuzz_sort_score',
                'fuzz_set_score',
                'uniq',
                'state_match',
                'zip_match',
                #'domain_match',
                'id_l count']

#training_data = pd.read_table('training_data_utf.txt')
X = training_data[feature_cols]
y = training_data['is_match']

kf = cross_validation.KFold(len(X), n_folds=10, shuffle=True)

recall_scores = []
precision_scores = []
f1_scores = []
n= 0
print("~~~~ CROSS VALIDATION each fold ~~~~")
for train_index, test_index in kf:
    lm = LogisticRegression().fit(X.iloc[train_index], y.iloc[train_index])
    recall_scores.append(metrics.recall_score(y.iloc[test_index], lm.predict(X.iloc[test_index])))
    precision_scores.append(metrics.precision_score(y.iloc[test_index], lm.predict(X.iloc[test_index])))
    f1_scores.append(metrics.f1_score(y.iloc[test_index], lm.predict(X.iloc[test_index])))
    n+=1
    print('Model', n)
    print('Recall:', recall_scores[n-1])
    print('Precision:', precision_scores[n-1])
    print('F1-Score:', f1_scores[n-1])

print("~~~~ SUMMARY OF CROSS VALIDATION ~~~~")
print('Mean recall scores for all folds:', np.mean(recall_scores))
print('Mean precision scores for all folds:', np.mean(precision_scores))
print('Mean F1 scores for all folds:', np.mean(f1_scores))

~~~~ CROSS VALIDATION each fold ~~~~
('Model', 1)
('Recall:', 0.8586956521739131)
('Precision:', 0.8186528497409327)
('F1-Score:', 0.8381962864721486)
('Model', 2)
('Recall:', 0.8842105263157894)
('Precision:', 0.9130434782608695)
('F1-Score:', 0.8983957219251336)
('Model', 3)
('Recall:', 0.9132947976878613)
('Precision:', 0.8729281767955801)
('F1-Score:', 0.8926553672316384)
('Model', 4)
('Recall:', 0.8983050847457628)
('Precision:', 0.8457446808510638)
('F1-Score:', 0.8712328767123287)
('Model', 5)
('Recall:', 0.8235294117647058)
('Precision:', 0.8700564971751412)
('F1-Score:', 0.8461538461538461)
('Model', 6)
('Recall:', 0.8602150537634409)
('Precision:', 0.8465608465608465)
('F1-Score:', 0.8533333333333333)
('Model', 7)
('Recall:', 0.9052631578947369)
('Precision:', 0.8514851485148515)
('F1-Score:', 0.8775510204081632)
('Model', 8)
('Recall:', 0.8852459016393442)
('Precision:', 0.8950276243093923)
('F1-Score:', 0.8901098901098902)
('Model', 9)
('Recall:', 0.8609625668449198)
('Prec